**TITRE**

In [2]:
import math

class AbstractRule:
    
    def _set_grammar(self,gram):
        self._grammar = gram
        
    def valuation(self):
        raise NotImplementedError
        
    def count(self):
        raise NotImplementedError
        
    def listR(self,i):
        raise NotImplementedError

        
#Exception triggered when a rule uses a literal not in the grammar
class UnknownLiteralError(Exception):
    
    def __init__(self,literal):
        self.literal = literal

        
class ConstructorRule(AbstractRule):
    
    def __init__(self, *args):
        self._parameters = args
        #print (self._parameters[0])
        #print (self._parameters[1])
        self._valuation = math.inf

    def valuation(self):
        return self._valuation
    
    #The Grammar is supposed non ambiguous
    #check if all literals used in the rule are parts of the grammar
    def _verif_rule(self):
        try:
            self._grammar[self._parameters[0]]
        except KeyError:
            raise UnknownLiteralError(self._parameters[0])
        try:
            self._grammar[self._parameters[1]]
        except KeyError:
            raise UnknownLiteralError(self._parameters[1])

    #Return true if there was no update
    def _update_valuation(self):
            #print("Valuation : Constructor")
            #print("old :"+str(self._valuation))
            self._old_val = self._valuation
            self._valuation = self._calc_valuation()
            #print("new :"+str(self._valuation))
            #print(self._old_val == self._valuation)
            return (self._old_val == self._valuation)

        
class UnionRule(ConstructorRule):
    
    def __init__(self,fst,snd):
        #print ("union")
        super().__init__(fst,snd)
        
    def _calc_valuation(self):
        #print ("union")
        return min(self._grammar[self._parameters[0]].valuation(),
                   self._grammar[self._parameters[1]].valuation())
    
    def count(self, i):
        #print (self._parameters[0])
        #print (self._parameters[1])
        #print (self._grammar[self._parameters[0]].count(i))
        #print (self._grammar[self._parameters[1]].count(i))
        return self._grammar[self._parameters[0]].count(i) + self._grammar[self._parameters[1]].count(i)
    
    def listR(self, i):
        return self._grammar[self._parameters[0]].listR(i) + self._grammar[self._parameters[1]].listR(i)
    
        
class ProductRule(ConstructorRule):
    
    def __init__(self,fst,snd,cons):
        #print("product")
        super().__init__(fst,snd)
        self._constructor = cons
        
    def _calc_valuation(self):
        #print("product")
        return (self._grammar[self._parameters[0]].valuation() +
               self._grammar[self._parameters[1]].valuation())
    
    def count(self, i):
        #print("count : product")
        res = 0
        valN1 = self._grammar[self._parameters[0]].valuation()
        valN2 = self._grammar[self._parameters[1]].valuation()
        #print (self._parameters[0])
        #print (self._parameters[1])
        #print("i :" + str(i))
        #print("valN1 :" + str(valN1))
        #print("valN2 :" + str(valN2))
        # WARNING
        # This might break some grammars (weird grammars with 2 different epsilon rules)
        # but avoid some infinite recursion (e.g. Dyck grammar)
        
        #if i == 0 and valN1 == 0 and valN2 == 0:
         #   return 1
        for k in range (valN1, i-valN2+1):
            #print("k : "+str(k))
            #print("l : "+str(i-k))
            res += self._grammar[self._parameters[0]].count(k) * self._grammar[self._parameters[1]].count(i-k)
            #print("res : "+str(res))
        return res
    
    def listR(self, i):
        #print("listR : product")
        res = []
        valN1 = self._grammar[self._parameters[0]].valuation()
        valN2 = self._grammar[self._parameters[1]].valuation()
        #print (self._parameters[0])
        #print (self._parameters[1])
        #print("i :" + str(i))
        #print("valN1 :" + str(valN1))
        #print("valN2 :" + str(valN2))
        # WARNING
        # This might break some grammars (weird grammars with 2 different epsilon rules)
        # but avoid some infinite recursion (e.g. Dyck grammar)
        
        #if i == 0 and valN1 == 0 and valN2 == 0:
         #   return 1
        for k in range (valN1, i-valN2+1):
            #print("k : "+str(k))
            #print("l : "+str(i-k))
            list0 = self._grammar[self._parameters[0]].listR(k)
            list1 = self._grammar[self._parameters[1]].listR(i-k)
            res += [self._constructor(x,y) for x in list0 for y in list1]
            
            #print("res : "+str(res))
        return res

    
class ConstantRule(AbstractRule):
    
    def __init__(self,obj):
        self._object = obj
        
    #We chose to put _update_valuation here too
    #to avoid testing the presence of the function
    #every time we want to call it on a rule
    def _update_valuation(self):
        return True

    
class EpsilonRule(ConstantRule):
    
    def __init__(self,obj):
        super().__init__(obj)
    
    def valuation(self):
        return 0
    
    def count(self, i):
        if i != 0:
            return 0
        else :
            return 1
        
    def listR(self,i):
        if i == 0:
            return [self._object]
        else:
            return []
        
    
    
    
class SingletonRule(ConstantRule):
    
    def __init__(self,obj):
        super().__init__(obj)
        
    def valuation(self):
        return 1
    
    def count(self, i):
        if i != 1:
            return 0
        else :
            return 1
        
    def listR(self,i):
        if i == 1:
            return [self._object]
        else:
            return []

#Triggered when a non terminal does not generate anything
class CircularGrammarError(Exception):
    
    def __init__(self,grammar,name):
        self.name = name
        self.grammar = grammar
        
    
def init_grammar(gram):
    #set the grammar for all rules
    for rule in gram.values() :
        rule._set_grammar(gram)
        #verifies every non-constant rule (check if all literal used in the rules are in the grammar)
        if isinstance(rule, ConstructorRule):
            rule._verif_rule()

    #As long as there's a change we update again
    #(we chose to make _update_valuation available
    #to any rule)
    while not all(rule._update_valuation() for rule in gram.values()):
        pass
    for name,rule in gram.items() :
        if rule.valuation() == math.inf :
            raise CircularGrammarError(gram,name)

In [3]:


#Timeout for the tests
import signal

class TimeoutError(Exception):
    pass

def timeout(signum,frame):
    raise TimeoutError
    
signal.signal(signal.SIGALRM, timeout)


#Common imports to all the tests
import operator

In [4]:
#Grammar definition for binary trees
treeGram = {"Tree" : UnionRule("Node","Leaf"),
            "Node" : ProductRule("Tree","Tree",
                                lambda a, b : "("+a+","+b+")"),
            "Leaf" : SingletonRule("Leaf")}
init_grammar(treeGram)

In [5]:
#tests for the binary trees grammar

#valuation

assert (treeGram["Tree"].valuation() == 1)
assert (treeGram["Node"].valuation() == 2)
assert (treeGram["Leaf"].valuation() == 1)

#count
"""
assert (treeGram["Leaf"].count(0) == 0)
assert (treeGram["Leaf"].count(1) == 1)
assert (treeGram["Leaf"].count(2) == 0)
assert (treeGram["Tree"].count(0) == 0)
assert (treeGram["Tree"].count(1) == 1)
assert (treeGram["Tree"].count(2) == 1)
assert (treeGram["Tree"].count(3) == 2)
assert (treeGram["Tree"].count(4) == 5)
assert (treeGram["Tree"].count(5) == 14)
assert (treeGram["Tree"].count(6) == 42)
assert (treeGram["Tree"].count(7) == 132)
assert (treeGram["Tree"].count(8) == 429)
assert (treeGram["Tree"].count(9) == 1430)
"""

#listR
assert (treeGram["Tree"].listR(0) == [])
assert (treeGram["Tree"].listR(1) == ["Leaf"])
assert (set(treeGram["Tree"].listR(2)) == set(["(Leaf,Leaf)"]))
assert (set(treeGram["Tree"].listR(3)) == set(["(Leaf,(Leaf,Leaf))",
                                      "((Leaf,Leaf),Leaf)"]))
assert (set(treeGram["Tree"].listR(4)) == set(["((Leaf,(Leaf,Leaf)),Leaf)",
                                      "(Leaf,((Leaf,Leaf),Leaf))",
                                      "((Leaf,Leaf),(Leaf,Leaf))",
                                      "(Leaf,(Leaf,(Leaf,Leaf)))",
                                      "(((Leaf,Leaf),Leaf),Leaf)"
                                  ]))

i = 0
try:
    while True:
        signal.alarm(1)
        assert (treeGram["Tree"].count(i) == len(treeGram["Tree"].listR(i)))
        signal.alarm(0)
        print(i)
        i+=1
except TimeoutError :
    print("timeout")

0
1
2
3
4
5
6
7
8
9
10
11
12
timeout


In [6]:
#Grammar definition for Fibonacci words
fiboGram = {"Fib" : UnionRule("Vide", "Cas1"),
            "Cas1" : UnionRule("CasAu", "Cas2"),
            "Cas2" : UnionRule("AtomB", "CasBAu"),
            "Vide" : EpsilonRule(""),
            "CasAu" : ProductRule("AtomA", "Fib",operator.add),
            "AtomA" : SingletonRule("A"),
            "AtomB" : SingletonRule("B"),
            "CasBAu" : ProductRule("AtomB", "CasAu",operator.add)}
init_grammar(fiboGram)

In [7]:
#tests for the fibonacci words grammar

#valuation

assert (fiboGram["Fib"].valuation() == 0)
assert (fiboGram["Cas1"].valuation() == 1)
assert (fiboGram["Cas2"].valuation() == 1)
assert (fiboGram["Vide"].valuation() == 0)
assert (fiboGram["CasAu"].valuation() == 1)
assert (fiboGram["AtomA"].valuation() == 1)
assert (fiboGram["AtomB"].valuation() == 1)
assert (fiboGram["CasBAu"].valuation() == 2)

#count



#listR

assert (fiboGram["Fib"].listR(0) == [""])
assert (set(fiboGram["Fib"].listR(1)) == set(["A","B"]))
assert (set(fiboGram["Fib"].listR(2)) == set(["AA","AB","BA"]))
assert (set(fiboGram["Fib"].listR(3)) == set(["AAA","AAB","ABA","BAA","BAB"]))
assert (set(fiboGram["Fib"].listR(4)) == set(["AAAA","AAAB","AABA","ABAA","ABAB","BAAA",
                                              "BAAB","BABA"]))

i = 0
try:
    while True:
        signal.alarm(1)
        assert (fiboGram["Fib"].count(i) == len(fiboGram["Fib"].listR(i)))
        signal.alarm(0)
        print(i)
        i+=1
except TimeoutError :
    print("timeout")
    
i = 0
try:
    while True:
        signal.alarm(1)
        assert (fiboGram["Cas1"].count(i) == len(fiboGram["Cas1"].listR(i)))
        signal.alarm(0)
        print(i)
        i+=1
except TimeoutError :
    print("timeout")
    
i = 0
try:
    while True:
        signal.alarm(1)
        assert (fiboGram["Cas2"].count(i) == len(fiboGram["Cas2"].listR(i)))
        signal.alarm(0)
        print(i)
        i+=1
except TimeoutError :
    print("timeout")
    
i = 0
try:
    while True:
        signal.alarm(1)
        assert (fiboGram["CasBAu"].count(i) == len(fiboGram["CasBAu"].listR(i)))
        signal.alarm(0)
        print(i)
        i+=1
except TimeoutError :
    print("timeout")
    
i = 0
try:
    while True:
        signal.alarm(1)
        assert (fiboGram["CasAu"].count(i) == len(fiboGram["CasAu"].listR(i)))
        signal.alarm(0)
        print(i)
        i+=1
except TimeoutError :
    print("timeout")

0
1
2
3
4
5
6
7
8
9
10
11
12
timeout
0
1
2
3
4
5
6
7
8
9
10
11
12
timeout
0
1
2
3
4
5
6
7
8
9
10
11
12
13
timeout
0
1
2
3
4
5
6
7
8
9
10
11
12
13
timeout
0
1
2
3
4
5
6
7
8
9
10
11
12
13
timeout


In [8]:
#Grammar definition for all the words with A and B letters
abGram = {"Words" : UnionRule("Vide","Cas1"),
          "Cas1" : UnionRule("CasAu","CasBu"),
          "AtomA" : SingletonRule("A"),
          "AtomB" : SingletonRule("B"),
          "CasAu" : ProductRule("AtomA", "Words","".join),
          "CasBu" : ProductRule("AtomB", "Words", "".join),
          "Vide" : EpsilonRule(""),
          }
init_grammar(abGram)

In [9]:
# tests for the (a,b) words grammar

#valuation
#TODO


In [10]:
#Grammar definition for Dyck words
dyckGram = {"Dyck" : UnionRule("Vide","Cas1"),
            "Cas1" : UnionRule("CasG","CasDouble"),
            "Vide" : EpsilonRule(""),
            "AtomG" : SingletonRule("("),
            "AtomD" : SingletonRule(")"),
            "CasG" : ProductRule("AtomG","CasD", "".join),
            "CasD" : ProductRule("Dyck","AtomD", "".join),
            "CasDouble" : ProductRule("Dyck","Dyck", "".join),
            }
init_grammar(dyckGram)

In [11]:
# tests for the Dyck words grammar

#valuation

assert (dyckGram["Dyck"].valuation() == 0)
assert (dyckGram["Cas1"].valuation() == 0)
assert (dyckGram["Vide"].valuation() == 0)
assert (dyckGram["AtomG"].valuation() == 1)
assert (dyckGram["AtomD"].valuation() == 1)
assert (dyckGram["CasG"].valuation() == 2)
assert (dyckGram["CasD"].valuation() == 1)
assert (dyckGram["CasDouble"].valuation() == 0)

#count

assert (dyckGram["Dyck"].count(0) == 1)
assert (dyckGram["Dyck"].count(1) == 1)
assert (dyckGram["Dyck"].count(2) == 2)
assert (dyckGram["Dyck"].count(3) == 5)
assert (dyckGram["Dyck"].count(4) == 5)
assert (dyckGram["Dyck"].count(5) == 14)
assert (dyckGram["Dyck"].count(6) == 42)
assert (dyckGram["Dyck"].count(7) == 132)
assert (dyckGram["Dyck"].count(8) == 429)
assert (dyckGram["Dyck"].count(9) == 1430)

RecursionError: maximum recursion depth exceeded

In [12]:
#Grammar definition for the (A,B) words without 3 times the same letter in a row
ab3Gram = {"Words" : UnionRule("Vide","Cas1"),
              "Cas1" : UnionRule("CasA","CasB"),
              "CasA" : ProductRule("AtomA","CasA1","".join),
              "CasA1" : UnionRule("CasB","CasA2"),
              "CasA2" : UnionRule("CasAB","Vide"),
              "CasB" : ProductRule("AtomB","CasB1","".join),
              "CasB1" : UnionRule("CasA","CasB2"),
              "CasB2" : UnionRule("CasBA","Vide"),
              "CasAB" : ProductRule("AtomA","CasAB1","".join),
              "CasAB1" : UnionRule("CasB","Vide"),
              "CasBA" : ProductRule("AtomB","CasBA1","".join),
              "CasBA1" : UnionRule("CasA","Vide"),
              "Vide" : EpsilonRule(""),
              "AtomA" : SingletonRule("A"),
              "AtomB" : SingletonRule("B")
             }

init_grammar(ab3Gram)

In [13]:
# tests for the (A,B) words without 3 times the same letter in a row grammar

#valuation
assert (ab3Gram["Words"].valuation() == 0)
assert (ab3Gram["Cas1"].valuation() == 1)
assert (ab3Gram["CasA"].valuation() == 1)
assert (ab3Gram["CasA1"].valuation() == 0)
assert (ab3Gram["CasA2"].valuation() == 0)
assert (ab3Gram["CasB"].valuation() == 1)
assert (ab3Gram["CasB1"].valuation() == 0)
assert (ab3Gram["CasB2"].valuation() == 0)
assert (ab3Gram["CasAB"].valuation() == 1)
assert (ab3Gram["CasAB1"].valuation() == 0)
assert (ab3Gram["CasBA"].valuation() == 1)
assert (ab3Gram["CasBA1"].valuation() == 0)
assert (ab3Gram["Vide"].valuation() == 0)
assert (ab3Gram["AtomA"].valuation() == 1)


In [14]:
#Grammar of all the words{A,B} where Card(A) = Card(B)
GramABequal = { "ABequal"   : UnionRule("Vide", "mot"),
                "mot"       : ProductRule("casBefore","casMiddle",operator.add),
                "casBefore" : ProductRule("casDouble","ABequal",operator.add),
                "casMiddle" : ProductRule("casAWB","casBWA",operator.add),
                "casDouble" : ProductRule("casAB","casBA",operator.add),
                "casAB"     : ProductRule("AtomA","AtomB",operator.add),
                "casBA"     : ProductRule("AtomB","AtomA",operator.add),
                "casAWB"    : ProductRule("AtomA","casWB",operator.add),
                "casBWA"    : ProductRule("AtomB", "casWA",operator.add),
                "casWA"     : ProductRule("ABequal","AtomA",operator.add),
                "casWB"     : ProductRule("ABequal","AtomB",operator.add),
                "AtomA"     : SingletonRule("A"),
                "AtomB"     : SingletonRule("B"),
                "Vide"      : EpsilonRule("")}

init_grammar(GramABequal)

In [16]:
#Grammar of Palyndromes{A,B,C}
PalindromeABC = {   "Palyn"     : UnionRule("Vide", "mot"),
                    "mot"       : UnionRule("AmotA","BCmotBC"),
                    "BCmotBC"   : UnionRule("BmotB","CmotC"),
                    "AmotA"     : ProductRule("AtomA","motA",operator.add),
                    "motA"      : ProductRule("Palyn","AtomA",operator.add),
                    "BmotB"     : ProductRule("AtomB","motB",operator.add),
                    "motB"      : ProductRule("Palyn","AtomB",operator.add),
                    "CmotC"     : ProductRule("AtomC","motC",operator.add),
                    "motC"      : ProductRule("Palyn","AtomC",operator.add),
                    "AtomA"     : SingletonRule("A"),
                    "AtomB"     : SingletonRule("B"),
                    "AtomC"     : SingletonRule("C"),
                    "Vide"      : EpsilonRule("")}

init_grammar(PalindromeABC)

CircularGrammarError: ({'Palyn': <__main__.UnionRule object at 0x7fe9bc2f8240>, 'mot': <__main__.UnionRule object at 0x7fe9bc2f81d0>, 'BCmotBC': <__main__.UnionRule object at 0x7fe9bc2f8278>, 'AmotA': <__main__.ProductRule object at 0x7fe9bc2f82e8>, 'motA': <__main__.ProductRule object at 0x7fe9bc2f8358>, 'BmotB': <__main__.ProductRule object at 0x7fe9bc2f8390>, 'motB': <__main__.ProductRule object at 0x7fe9bc2f83c8>, 'CmotC': <__main__.ProductRule object at 0x7fe9bc2f8400>, 'motC': <__main__.ProductRule object at 0x7fe9bc2f8438>, 'AtomA': <__main__.SingletonRule object at 0x7fe9bc2f8470>, 'AtomB': <__main__.SingletonRule object at 0x7fe9bc2f84a8>, 'AtomC': <__main__.SingletonRule object at 0x7fe9bc2f84e0>, 'Vide': <__main__.EpsilonRule object at 0x7fe9bc2f8518>}, 'mot')